## Create an Agent

### Example of an Agent
- it doesn't have tools yet, so getting weather is not feasible, but sure it can get a joke

In [ ]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.monitor.opentelemetry import configure_azure_monitor

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["MODEL_DEPLOYMENT_NAME"]


In [ ]:
# Create a logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

# Clear existing handlers to avoid duplicate logs
if logger.hasHandlers():
    logger.handlers.clear()

# Create handler that outputs to notebook
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

# Example usage
logger.info("Logger is now printing to the notebook!")

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

In [ ]:

agent_name = os.environ["AZURE_AI_AGENT_NAME_01"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")

In [ ]:


# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

if not agent:
    logger.info(f"Agent with name '{agent_name}' not found, creating a new agent.")

    agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name=agent_name,
            instructions="You are a helpful agent who does not have tools available, but can rely on your knowledge to answer questions.",
            tools=None,
        )
    logger.info(f"Created agent, ID: {agent.id}")
else:
    logger.info(f"Found agent by name '{agent_name}', ID={agent.id}")


thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="what is the weather in London?",  # Message content
    )
print(f"Created message, ID: {message['id']}")





In [ ]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print(agent.name)
print(agent.id)

In [ ]:
import json
from IPython.display import display, Markdown

json_string = json.dumps(run, default=str, indent=4)

display(Markdown(f"```json\n{json_string}\n```"))


In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    print(f"Role: {message.role}, : {message.content[0]['text']['value'] if message.content else 'No content'}")

In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    json_string = json.dumps(message, default=str, indent=4)
    display(Markdown(f"```json\n{json_string}\n```"))
    display(Markdown(f"---"))


## Let's ask it for a joke - notice all messages are available, since we are using the existing thread

In [ ]:
# Create message to thread
message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="ok fair, what about giving me a joke.",  # Message content
    )
print(f"Created message, ID: {message['id']}")

run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print(agent.name)
print(agent.id)

if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    print(f"Role: {message.role}, : {message.content[0]['text']['value'] if message.content else 'No content'}")
